# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# B

from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
# load data from database
engine = create_engine('sqlite:///ETL_PREPARATION.db')
df = pd.read_sql_table("ETL_PREPARATION",engine)
x = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [23]:
#B

def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
# B

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
# B

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42)
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
#B
y_pred = pipeline.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i], target_names=category_names), '............................................')




related :
                        precision    recall  f1-score   support

               related       0.63      0.34      0.44      1553
               request       0.82      0.94      0.87      4991
                 offer       0.64      0.34      0.44        53

           avg / total       0.77      0.79      0.77      6597
 ............................................
request :
                        precision    recall  f1-score   support

               related       0.88      0.98      0.93      5446
               request       0.84      0.39      0.53      1151

           avg / total       0.88      0.88      0.86      6597
 ............................................
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6568
               request       1.00      0.03      0.07        29

           avg / total       1.00      1.00      0.99      6597
 .........................................

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = eval_metrics(y_train.loc[:,column].values,y_pred_train[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df   

,Accuracy,Precision,Recall,f1 score
0,0.990449,1.0,0.990449,0.990449
1,0.985295,1.0,0.985295,0.985295
2,0.998737,1.0,0.998737,0.998737
3,0.980949,1.0,0.980949,0.980949
4,0.987013,1.0,0.987013,0.987013
5,0.991207,1.0,0.991207,0.991207
6,0.993987,1.0,0.993987,0.993987
7,0.995907,1.0,0.995907,0.995907
8,0.994239,1.0,0.994239,0.994239
9,1.000000,1.0,1.000000,1.000000


In [12]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = eval_metrics(y_test.loc[:,column].values,y_pred_test[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

,Accuracy,Precision,Recall,f1 score
0,0.810217,1.0,0.810217,0.810217
1,0.879794,1.0,0.879794,0.879794
2,0.995756,1.0,0.995756,0.995756
3,0.728665,1.0,0.728665,0.728665
4,0.922692,1.0,0.922692,0.922692
5,0.955283,1.0,0.955283,0.955283
6,0.973624,1.0,0.973624,0.973624
7,0.981810,1.0,0.981810,0.981810
8,0.966955,1.0,0.966955,0.966955
9,1.000000,1.0,1.000000,1.000000


In [13]:
train_results_df.mean()

Accuracy     0.992566
Precision    1.000000
Recall       0.992566
f1 score     0.992566
dtype: float64

In [14]:
test_results_df.mean()

Accuracy     0.942571
Precision    1.000000
Recall       0.942571
f1 score     0.942571
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [34]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f0efe5b3d08>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [35]:
parameters = {
    #'vect__ngram_range': ((1, 1),(1,2)),
    #'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000),
    #'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 150],
    'clf__estimator__min_samples_split': [2, 3, 4],
    
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)

In [36]:
#MultiOutputClassifier(KNeighborsClassifier()).fit(X, y)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=14.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=14.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total=14.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=150 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total=27.5min
[CV] clf__estimator__min_samples_split=2, clf__est

[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed: 187.7min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'clf__estimator__n_estimators': [50, 100, 150], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [37]:
cv.grid_scores_

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.23826, std: 0.00870, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50},
 mean: 0.24347, std: 0.00660, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100},
 mean: 0.24281, std: 0.00639, params: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 150},
 mean: 0.23821, std: 0.00394, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 50},
 mean: 0.24089, std: 0.00716, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 100},
 mean: 0.24362, std: 0.00836, params: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 150},
 mean: 0.23361, std: 0.00789, params: {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 50},
 mean: 0.24221, std: 0.00821, params: {'clf__estimator__min_samples_split': 4, 'clf__estimator__n_estimators': 100},
 mean: 0.23917, std: 0.00509, params: {'clf__estimator__min_samples

In [38]:
#best parameter by score

print(cv.best_params_)

{'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 150}


In [39]:
# new model 

optimized_model = cv.best_estimator_
print (cv.best_estimator_)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [40]:
y_pred = optimized_model.predict(X_test)

for i in range(36):
    print(y_test.columns[i], ':')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i], target_names=category_names), '............................................')



related :
                        precision    recall  f1-score   support

               related       0.76      0.26      0.39      1553
               request       0.80      0.97      0.88      4991
                 offer       0.59      0.25      0.35        53

           avg / total       0.79      0.80      0.76      6597
 ............................................
request :
                        precision    recall  f1-score   support

               related       0.89      0.99      0.94      5446
               request       0.90      0.44      0.59      1151

           avg / total       0.89      0.89      0.88      6597
 ............................................
offer :
                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6568
               request       1.00      0.03      0.07        29

           avg / total       1.00      1.00      0.99      6597
 .........................................

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [46]:
with open('MLclassifier.pkl', 'wb') as file:
    pickle.dump(optimized_model, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.